In [1]:
import findspark
findspark.find()
findspark.init()
import pyspark
findspark.find()

'/opt/spark/python/pyspark'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('finalProject').getOrCreate()

22/05/04 20:16:45 WARN Utils: Your hostname, eternal resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp0s20f3)
22/05/04 20:16:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/04 20:16:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
Dataset = spark.read.load("/home/eternal/bigData/finalProject/Data Set - Sheet1.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )

In [4]:
Dataset.show()

+------+-------+----------------+--------------------+----------+
|SL No.| Emp ID|RFID Card Number|       Employee Name|   Contact|
+------+-------+----------------+--------------------+----------+
|     1|4000001|         4000001|        Shakil Ahmed|1812345678|
|     2|4000002|         4000002|        Shatabdi Roy|1812345678|
|     3|4000003|         4000003|    Md. Pieash Ahmed|         0|
|     4|4000004|         4000004|        Fahima Islam|1812345678|
|     5|4000005|         4000005|  Tanjina Akter Jame|1812345678|
|     6|4000006|         4000006|    Afrin Jahan Urme|1812345678|
|     7|4000007|         4000007|      Anika Tabassum|1812345678|
|     8|4000008|         4000008|     Afsana Alam Mim|1812345678|
|     9|4000009|         4000009| Sayma Islam Sochona|1812345678|
|    10|4000010|         4000010|A. K. M. Tasnim Alam|1812345678|
|    11|4000011|         4000011|   Md Regone Hossain|1812345678|
|    12|4000012|         4000012| Silvia Kh. Shuvessa|1812345678|
|    13|40

In [5]:
#Importing all the necessary library 

from pyspark.sql import functions as F
from pyspark.sql.functions import rand,when
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from random import randint
import random
import string


In [23]:
# Random UserId generator with string and number 

def ran(r):
    return "".join([random.choice(string.ascii_lowercase + string.digits) for i in range(r)])


def randIdGenerator():
    return "".join(ran(8)+"-"+ran(4)+"-"+ran(4)+"-"+ran(12))
    
randIdGenerator = F.udf(randIdGenerator, StringType())

In [24]:

# Random Date Of Birth Generator

def randDateOfBirth():
    return "".join(str(random.randint(1969, 1998)))+"-"+"".join(str(random.randint(1, 12)).zfill(2))+"-"+"".join(str(random.randint(1, 28)).zfill(2))

randDateOfBirth = F.udf(randDateOfBirth , StringType())

In [25]:

#lon-lat-generator

def lonlat():
    return float("{0:.5f}".format(random.uniform(20.11, 100.1)))


lonlat = F.udf(lonlat, StringType())

In [26]:
#Assingning 'N' if the number is 0

def otpV(contact_number):
    if contact_number == '0':
        return 'N'
    else:
        return 'Y'
    

otpV = F.udf(otpV, StringType())

In [ ]:
#Table 1

In [27]:

cdf = Dataset.withColumn('user_id',randIdGenerator())\
    .withColumn("split_list", F.split(F.col("Employee Name"), " ")).withColumn("first_name", col("split_list")[0])\
    .withColumn("last_name", col("split_list")[F.size("split_list") - 1]).drop("split_list")\
    .withColumn('date_of_birth',randDateOfBirth())\
    .withColumn('address',F.array(F.lit("Dhaka, Bangladesh"),F.lit("Sylhet, Bangladesh"),F.lit("Chittagong, Bangladesh"),F.lit("Rangpur, Bangladesh"),).getItem((F.rand()*4).cast("int")))\
    .withColumn('lon',lonlat())\
    .withColumn('lat',random.uniform(F.col('lon'),10))\
    .withColumn('created_date',lit('2022-03-12'))\
    .withColumn('updated_date',current_date())\
    .withColumn("is_otp_verified",otpV('Contact'))\
    .withColumn('contact_number',col('Contact')).drop('SL No.','Emp ID','RFID Card Number','Employee Name')

users_personalinfo = cdf.select('user_id','first_name','last_name','date_of_birth','contact_number','address','lat','lon','created_date','updated_date','is_otp_verified')

/opt/spark/python/pyspark/sql/column.py:322: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [28]:
users_personalinfo.limit(10).toPandas()

,user_id,first_name,last_name,date_of_birth,contact_number,address,lat,lon,created_date,updated_date,is_otp_verified
0,qxduhdo6-sr4y-r565-aczxcao9r5ja,Shakil,Ahmed,1994-10-13,1812345678,"Chittagong, Bangladesh",58.802606,68.41463,2022-03-12,2022-05-04,Y
1,a9gkiha8-txa4-j9b3-b6mynpvvp836,Shatabdi,Roy,1985-11-21,1812345678,"Sylhet, Bangladesh",59.380429,69.10626,2022-03-12,2022-05-04,Y
2,u2hfniem-cyka-q8bd-ivgjiskuhr4p,Md.,Ahmed,1971-03-23,0,"Chittagong, Bangladesh",26.902584,30.23167,2022-03-12,2022-05-04,N
3,785assei-c51d-r9wl-10phhplib43k,Fahima,Islam,1994-04-06,1812345678,"Chittagong, Bangladesh",65.771652,76.75628,2022-03-12,2022-05-04,Y
4,3lrjwx5u-xz3t-vcya-tj7z63yn6tz5,Tanjina,Jame,1985-10-06,1812345678,"Dhaka, Bangladesh",81.848066,95.99906,2022-03-12,2022-05-04,Y
5,ob5r2kpp-qvnu-gjhs-7dw3y5froww8,Afrin,Urme,1976-10-11,1812345678,"Sylhet, Bangladesh",43.150149,49.67931,2022-03-12,2022-05-04,Y
6,ban2vdgg-341g-jdhk-g85sk8nhiwe0,Anika,Tabassum,1994-12-19,1812345678,"Dhaka, Bangladesh",62.497115,72.8368,2022-03-12,2022-05-04,Y
7,pvev1l6x-u4gx-7dg8-p06r6dzeyl73,Afsana,Mim,1969-10-12,1812345678,"Rangpur, Bangladesh",18.566530,20.25377,2022-03-12,2022-05-04,Y
8,3ev3jr1s-s8i4-oycy-n1x7ws9nh6gf,Sayma,Sochona,1974-10-15,1812345678,"Sylhet, Bangladesh",58.525612,68.08308,2022-03-12,2022-05-04,Y
9,g7vb3pdd-0dqh-4vne-c6ir5abam4zt,A.,Alam,1993-11-26,1812345678,"Rangpur, Bangladesh",31.292900,35.48669,2022-03-12,2022-05-04,Y


In [36]:
users_personalinfo.write.option("header",True).csv("/home/eternal/bigData/finalProject/DataSet/users_personalinfo")
print('Success')

Success


In [53]:
datasetTable1 = spark.read.load("/home/eternal/bigData/finalProject/DataSet/users_personalinfo/part-00000-81eaa0eb-0a7f-4b96-b048-11a09d8f2d2a-c000.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )
datasetTable1.toPandas().head(20)

,user_id,first_name,last_name,date_of_birth,contact_number,address,lat,lon,created_date,updated_date,is_otp_verified
0,yzscg2ft-2np9-70ci-z1vnjh5boimt,Shakil,Ahmed,1985-04-27,1812345678,"Dhaka, Bangladesh",55.36257268487463,100.03733,2022-03-12,2022-04-06,Y
1,c3rocgh0-gcs7-jm0h-k7hszjupk4td,Shatabdi,Roy,1976-11-03,1812345678,"Rangpur, Bangladesh",34.30316984256811,58.23784,2022-03-12,2022-04-06,Y
2,gshv586c-c972-t0cp-j8kkmzmd7z1k,Md.,Ahmed,1987-09-15,0,"Sylhet, Bangladesh",16.352601031981536,22.60888,2022-03-12,2022-04-06,N
3,t7y7njd6-d973-w4j9-xhkenvoywbcq,Fahima,Islam,1988-09-28,1812345678,"Rangpur, Bangladesh",38.837747880288084,67.23824,2022-03-12,2022-04-06,Y
4,o3vqa7x1-eyg5-016l-0hc9na18qkcm,Tanjina,Jame,1974-12-26,1812345678,"Dhaka, Bangladesh",37.057405554823575,63.70455,2022-03-12,2022-04-06,Y
5,gql3arpr-u6b3-lkjg-cm3kxhqviasn,Afrin,Urme,1997-09-18,1812345678,"Dhaka, Bangladesh",15.57472370351634,21.06492,2022-03-12,2022-04-06,Y
6,yc6csh7s-xmjf-qslg-620id8jcrgde,Anika,Tabassum,1989-09-03,1812345678,"Sylhet, Bangladesh",42.31219372819538,74.13445,2022-03-12,2022-04-06,Y
7,7ci6hvbr-rojr-0paj-4da6h5vn4d9w,Afsana,Mim,1985-05-25,1812345678,"Dhaka, Bangladesh",17.89357865968585,25.66747,2022-03-12,2022-04-06,Y
8,npqqq41s-tnza-j5c2-k490g5dv4d59,Sayma,Sochona,1985-12-23,1812345678,"Rangpur, Bangladesh",28.345759876206678,46.41335,2022-03-12,2022-04-06,Y
9,w4e6t48q-teg1-gad2-63mj1amc7vqp,A.,Alam,1971-07-21,1812345678,"Chittagong, Bangladesh",25.900698107178975,41.5603,2022-03-12,2022-04-06,Y


In [ ]:
#Table 2

In [10]:

#Assigning PostCode from City

def postCode(City):
    if City == 'Dhaka':
        return '1209'
    elif City == 'Sylhet':
        return '3352'
    elif City == 'Chittagong':
        return '4620'
    else:
        return '5201'
    
    
# Set address by Specific city
def address(City):
    if City == 'Dhaka':
        return 'Dhanmondi,Dhaka,Bangladesh'
    elif City == 'Sylhet':
        return 'Hobiganj,Sylhet,Bangladesh'
    elif City == 'Chittagong':
        return 'Bandarban,Chittagong,Bangladesh'
    else:
        return 'Dinajpur,Bangladesh'


address = F.udf(address, StringType())
postCode = F.udf(postCode, StringType())

In [55]:
cdf = Dataset.withColumn('user_id',randIdGenerator())\
    .withColumn('country',lit('Bangladesh'))\
    .withColumn("City",F.array(F.lit("Dhaka"),F.lit("Sylhet"),F.lit("Chittagong"),F.lit("Rangpur"),).getItem((F.rand()*4).cast("int")))\
    .withColumn('postcode',postCode('City'))\
    .withColumn('address',address('City'))\
    .withColumn('created_date',lit('2021-09-12'))\
    .withColumn('updated_date',current_date())\
    .withColumnRenamed('Contact','contact')\
    .withColumn('currency',lit('BDT')).drop('SL No.','Emp ID','RFID Card Number','Employee Name')

users_billinginfo = cdf.select("user_id","address","city","postcode","country","contact","currency","created_date","updated_date")
    

/opt/spark/python/pyspark/sql/column.py:322: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [56]:
users_billinginfo.limit(10).toPandas()

# user_id, address, city, postcode, country, contact, currency, created_date, updated_date

,user_id,address,city,postcode,country,contact,currency,created_date,updated_date
0,xh7yiw6x-djg3-w9rf-j2hxbp9h9pua,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
1,igp9eg0u-wqw2-jcox-9qpjporw7fk4,"Dhanmondi,Dhaka,Bangladesh",Dhaka,1209,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
2,7gff5jkj-pegw-vm5k-2tnjqembuju9,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,0,BDT,2021-09-12,2022-04-06
3,b64sb634-f3uh-9x4a-x4iy3g35hlrw,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
4,yo3hsqxr-tif3-kvc8-3p4w6kehxfze,"Hobiganj,Sylhet,Bangladesh",Sylhet,3352,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
5,io0pc3li-e4sf-5sbe-a45eh1j4niu9,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
6,ktqmzfy4-2xu9-d420-i6ao0740dkq5,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
7,ehteepz1-tufg-yac0-9qcdzoekwypt,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
8,6xve50dg-dvya-9x7o-gbe19sre7gq9,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
9,wbzxhl92-uiav-j6ko-2v88qpdqgawb,"Dinajpur,Bangladesh",Rangpur,5201,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06


In [57]:
users_billinginfo.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- country: string (nullable = false)
 |-- contact: string (nullable = true)
 |-- currency: string (nullable = false)
 |-- created_date: string (nullable = false)
 |-- updated_date: date (nullable = false)



In [58]:
users_billinginfo.write.option("header",True).csv("/home/eternal/bigData/finalProject/DataSet/users_billinginfo")

In [7]:
datasetTable2 = spark.read.load("/home/eternal/bigData/finalProject/DataSet/users_billinginfo/part-00000-d8c2d438-9868-48ac-baae-a689d1988d40-c000.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )
datasetTable2.limit(10).toPandas()

,user_id,address,city,postcode,country,contact,currency,created_date,updated_date
0,4gc5gjfg-3ki7-i9rf-5g9x5i5tvuot,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
1,8ukdvgkf-sx0b-whuy-2t3l3cex41xm,"Dhanmondi,Dhaka,Bangladesh",Dhaka,1209,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
2,pnejbl5h-oya1-uivo-hkjez50dks1y,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,0,BDT,2021-09-12,2022-04-06
3,qk687pjk-zdsn-c7co-zggmajc35wtd,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
4,yxwnz3mc-5r5m-l8dh-5dzpr09xqdpu,"Hobiganj,Sylhet,Bangladesh",Sylhet,3352,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
5,ftsnmcn6-4myg-fh6d-7lzwaf8n2syr,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
6,jedwr8wc-mr4a-478j-un302luxowsj,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
7,m9dd30eg-p5li-m2p6-5cb70u60a1wc,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
8,0cds7edj-zuco-mhii-s4k8tardbfzm,"Bandarban,Chittagong,Bangladesh",Chittagong,4620,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06
9,nag8e0vk-e343-gsco-yaa97n8r66ej,"Dinajpur,Bangladesh",Rangpur,5201,Bangladesh,1812345678,BDT,2021-09-12,2022-04-06


In [99]:
#table 3

In [23]:
cdf = Dataset.withColumn('unique_user_id',round(rand()*100300,1000).cast("int"))\
    .withColumn("user_id",randIdGenerator())\
    .withColumn("activated",F.lit('1'))\
    .withColumn("tag",round(rand()*123456,1).cast("int"))\
    .withColumn('created_date',lit('2020-11-12'))\
    .withColumn('updated_date',current_date())\
    .withColumn('valid_until',F.array(F.lit("05/29/2025"),F.lit("08/17/2028"),F.lit("01/08/2023"),).getItem((F.rand()*3).cast("int")))\
    .withColumn('last_used',F.array(F.lit("08/19/2021"),F.lit("08/21/2021"),F.lit("06/22/2021"),).getItem((F.rand()*3).cast("int")))\
    .withColumn('reference_number',round(rand()*(12534-123)+125).cast("int"))\
    .withColumn('employee_code',col('reference_number'))\
    .drop('SL No.','Emp ID','RFID Card Number','Employee Name')

users_rfidinfo = cdf.select("user_id","tag","activated","created_date","updated_date","valid_until","last_used","reference_number","employee_code","unique_user_id")
   

/opt/spark/python/pyspark/sql/column.py:322: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [24]:
users_rfidinfo.toPandas().head(10)

,user_id,tag,activated,created_date,updated_date,valid_until,last_used,reference_number,employee_code,unique_user_id
0,rl5lvspx-6bbk-ru14-pcg6yxvd1e4y,15370,1,2020-11-12,2022-04-12,01/08/2023,06/22/2021,12203,12203,12592
1,l9xntq2o-5q44-fx63-uw09wiquajvu,79439,1,2020-11-12,2022-04-12,08/17/2028,06/22/2021,12275,12275,4044
2,aim84pdn-0j10-zob3-jvft1bwlcwbh,119545,1,2020-11-12,2022-04-12,05/29/2025,08/21/2021,11919,11919,55408
3,gm2ge4hz-vdmj-61bx-h4q2i5qyhlbd,10190,1,2020-11-12,2022-04-12,05/29/2025,08/21/2021,11249,11249,22118
4,znj2rmdn-whi0-7zyy-9jnqbxhd4jf0,59668,1,2020-11-12,2022-04-12,05/29/2025,08/19/2021,7010,7010,77776
5,9s0ezlgx-m6n8-fuk0-0w7o91mgdraz,18786,1,2020-11-12,2022-04-12,01/08/2023,08/21/2021,4544,4544,95612
6,03xwaxll-pvnq-k06e-84qiqrkdqi3a,44478,1,2020-11-12,2022-04-12,08/17/2028,08/19/2021,5434,5434,74001
7,9zxp1eng-hujy-nv4u-qt9xpkn7b5ll,71443,1,2020-11-12,2022-04-12,05/29/2025,08/21/2021,11876,11876,57323
8,0i6i9v68-r4j8-0zot-0rnaxcobpgy4,61589,1,2020-11-12,2022-04-12,05/29/2025,06/22/2021,11814,11814,43101
9,zqkhkwjo-vx82-r97f-9hn5ce7uclqm,37006,1,2020-11-12,2022-04-12,08/17/2028,08/21/2021,9843,9843,31321


In [25]:
users_rfidinfo.write.option("header",True).csv("/home/eternal/bigData/finalProject/DataSet/users_rfidinfo")

In [26]:
datasetTable3 = spark.read.load("/home/eternal/bigData/finalProject/DataSet/users_rfidinfo/part-00000-720adf36-16f4-4453-9a6b-bf67be710860-c000.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )
datasetTable3.toPandas().head(10)

,user_id,tag,activated,created_date,updated_date,valid_until,last_used,reference_number,employee_code,unique_user_id
0,7s0hczms-ftjy-mkc2-uxw96xhc43we,15370,1,2020-11-12,2022-04-12,01/08/2023,06/22/2021,12203,12203,12592
1,ooz53cs5-zu11-vs89-9r3x9n4aodui,79439,1,2020-11-12,2022-04-12,08/17/2028,06/22/2021,12275,12275,4044
2,o74twatg-4dvt-yk9a-20bxd2eecfhv,119545,1,2020-11-12,2022-04-12,05/29/2025,08/21/2021,11919,11919,55408
3,dlm9thbn-gqrp-yf04-wzorzyoc3ccv,10190,1,2020-11-12,2022-04-12,05/29/2025,08/21/2021,11249,11249,22118
4,qhttt0te-y7sw-tp2b-dkuauu6sce20,59668,1,2020-11-12,2022-04-12,05/29/2025,08/19/2021,7010,7010,77776
5,v9yiv3mq-iu2d-8hwe-xzjv1pu2my9n,18786,1,2020-11-12,2022-04-12,01/08/2023,08/21/2021,4544,4544,95612
6,6n66xgzv-59su-5v8x-j40hbd5i5rck,44478,1,2020-11-12,2022-04-12,08/17/2028,08/19/2021,5434,5434,74001
7,rlt1feag-adwj-pn4d-ligxk3vo9bn0,71443,1,2020-11-12,2022-04-12,05/29/2025,08/21/2021,11876,11876,57323
8,j6ttsasv-6s2f-i1nl-mg4g0tvpxlwc,61589,1,2020-11-12,2022-04-12,05/29/2025,06/22/2021,11814,11814,43101
9,3ix4ltaq-q2w2-vubp-7nxrzqubfbl2,37006,1,2020-11-12,2022-04-12,08/17/2028,08/21/2021,9843,9843,31321


In [ ]:
#Table 4

In [11]:
y = input("Enter Product Type Id: ")

Enter Product Type Id: 574


In [12]:

cdf = Dataset.withColumn('id',round(rand()*100300,1000).cast("int"))\
    .withColumn("quantity",round(rand()*(1250-240)+10).cast("int"))\
    .withColumn('created_date',lit('2021-05-29'))\
    .withColumn('updated_date',current_date())\
    .withColumn('product_type_id',lit(y))\
    .withColumn('rfid_id',randIdGenerator())\
    .withColumn('credit_type',lit('NORMAL'))\
    .withColumn('expire_date',F.array(F.lit("05/29/2025"),F.lit("08/17/2028"),F.lit("01/08/2023"),).getItem((F.rand()*3).cast("int"))).drop('SL No.','Employee Name','Contact')

users_rfiditem = cdf.select("id","quantity","created_date","updated_date","product_type_id","rfid_id","credit_type","expire_date")
  

/opt/spark/python/pyspark/sql/column.py:322: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [13]:
users_rfiditem.toPandas().head(10)

,id,quantity,created_date,updated_date,product_type_id,rfid_id,credit_type,expire_date
0,4942,658,2021-05-29,2022-04-12,574,u2pbu4bl-errn-tlbs-yi7tawgphcq5,NORMAL,05/29/2025
1,19605,353,2021-05-29,2022-04-12,574,kx0hyrue-8xwb-r5ne-siinw0ur73te,NORMAL,08/17/2028
2,29822,490,2021-05-29,2022-04-12,574,hopapou2-xgw8-44u8-c831u8c6shrl,NORMAL,01/08/2023
3,9319,730,2021-05-29,2022-04-12,574,g7bor9ge-ijo4-4e3e-z86w0hr71aq7,NORMAL,08/17/2028
4,22601,16,2021-05-29,2022-04-12,574,zavie6ff-09fd-hcjl-zbttt09j2xs0,NORMAL,01/08/2023
5,34065,114,2021-05-29,2022-04-12,574,5ilw02aw-9nze-rivk-7ockd2uqcgib,NORMAL,01/08/2023
6,68154,466,2021-05-29,2022-04-12,574,4lyj6lqy-fphc-8qv5-cksmb2x5vimy,NORMAL,05/29/2025
7,30975,752,2021-05-29,2022-04-12,574,aw4sp5i5-z9fb-qfld-ege3k7leywnb,NORMAL,01/08/2023
8,67023,138,2021-05-29,2022-04-12,574,aq43bpeu-noqh-sdgm-c4r15msc8f7t,NORMAL,01/08/2023
9,18245,154,2021-05-29,2022-04-12,574,2bnkd0gb-mx03-mbqu-12aj6ylfgit4,NORMAL,05/29/2025


In [14]:
#Dumping The created dataset as a csv file.

users_rfiditem.write.option("header",True).csv("/home/eternal/bigData/finalProject/DataSet/users_rfiditem")

In [15]:
#Opening the created file from folder
datasetTable4 = spark.read.load("/home/eternal/bigData/finalProject/DataSet/users_rfiditem/part-00000-6c029b21-5d9a-4ca2-9c94-c8d4cd5cf7b5-c000.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )
datasetTable4.toPandas().head(10)


,id,quantity,created_date,updated_date,product_type_id,rfid_id,credit_type,expire_date
0,4942,658,2021-05-29,2022-04-12,574,dlj01jgs-h2by-z75p-mzezs6j1r9fi,NORMAL,05/29/2025
1,19605,353,2021-05-29,2022-04-12,574,tt7320o8-3xbs-p95b-9meda6mqn2in,NORMAL,08/17/2028
2,29822,490,2021-05-29,2022-04-12,574,zhzy36hy-nx9k-fszs-aakw0qtfyenv,NORMAL,01/08/2023
3,9319,730,2021-05-29,2022-04-12,574,96vvubxl-2epz-3hae-ooaosovtstj8,NORMAL,08/17/2028
4,22601,16,2021-05-29,2022-04-12,574,f5khcg3j-t9ex-68gy-38nuwp82zq9s,NORMAL,01/08/2023
5,34065,114,2021-05-29,2022-04-12,574,htzkjd3u-hm1r-zdjf-hprsg3qnljxn,NORMAL,01/08/2023
6,68154,466,2021-05-29,2022-04-12,574,urqndiwn-wcz5-585o-e1tkoztcgjpk,NORMAL,05/29/2025
7,30975,752,2021-05-29,2022-04-12,574,xp1urqoh-d2og-4xp0-28mtgbk3s4m8,NORMAL,01/08/2023
8,67023,138,2021-05-29,2022-04-12,574,hsmad7wq-o49o-7k5p-2tcjoj6pbye6,NORMAL,01/08/2023
9,18245,154,2021-05-29,2022-04-12,574,23gsidon-aqm3-eh7o-uu26b5yp18o5,NORMAL,05/29/2025


In [ ]:
#Table 5 

In [17]:
inventory_type_id = input("Enter inventory type id: ")

Enter inventory type id: 574


In [18]:

cdf = Dataset.withColumn('id',round(rand()*100300,1000).cast("int"))\
    .withColumn('user_id',randIdGenerator())\
    .withColumn("per_day",round(rand()*9,5).cast("int"))\
    .withColumn("per_week",round(rand()*12,18).cast("int"))\
    .withColumn("per_month",round(rand()*12,28).cast("int"))\
    .withColumn('created_date',lit('2021-01-29'))\
    .withColumn('updated_date',current_date())\
    .withColumn('inventory_type_id',lit(inventory_type_id)).drop('Employee Name','Contact','Emp ID')

users_productlimit = cdf.select("id","per_day","per_week","per_month",
                                "created_date","updated_date","inventory_type_id","user_id")
  

In [19]:
users_productlimit.toPandas().head(10)


,id,per_day,per_week,per_month,created_date,updated_date,inventory_type_id,user_id
0,67294,2,2,10,2021-01-29,2022-04-10,574,g1hghdvs-mnc4-envl-0eyta639ep7i
1,67918,6,11,8,2021-01-29,2022-04-10,574,fyd9t08x-wzby-ylbt-0fu7nzf8fakv
2,77900,7,8,10,2021-01-29,2022-04-10,574,qviki5r5-5tcr-1nxo-mg49c3hwtrcl
3,59437,7,1,2,2021-01-29,2022-04-10,574,jqn2ya26-2ke8-rdq5-oel433j30b7q
4,14404,0,10,5,2021-01-29,2022-04-10,574,g7s6hl5w-34m1-ep4t-5291v2c99gxk
5,33804,1,11,9,2021-01-29,2022-04-10,574,9jrzklb9-h0sr-h7qx-ecwqldqs5uw4
6,55346,1,11,11,2021-01-29,2022-04-10,574,lpd65pa4-8i0y-rq34-ev70u4x0im71
7,42820,7,5,9,2021-01-29,2022-04-10,574,w2gvnlkl-4i61-emp9-mgjxhe723bzk
8,11067,4,11,10,2021-01-29,2022-04-10,574,dub6fisr-k2v8-h46q-aiz942xsj8bk
9,98507,3,10,4,2021-01-29,2022-04-10,574,ymz8157t-lvav-c2hg-zpuvkda3j50u


In [20]:

#Dumping The created dataset as a csv file.

users_productlimit.write.option("header",True).csv("/home/eternal/bigData/finalProject/DataSet/users_productlimit")

In [16]:

#Opening the created file from folder
datasetTable5 = spark.read.load("/home/eternal/bigData/finalProject/DataSet/users_productlimit/part-00000-25b0b47f-8d79-44e7-b55d-4edb9375f1ee-c000.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )
datasetTable5.toPandas().head(10)

,id,per_day,per_week,per_month,created_date,updated_date,inventory_type_id,user_id
0,67294,2,2,10,2021-01-29,2022-04-10,574,59ekghdz-qt0i-wx30-njs5ng9ul39a
1,67918,6,11,8,2021-01-29,2022-04-10,574,ug9jzfbh-b222-8put-zkxow2zioxxa
2,77900,7,8,10,2021-01-29,2022-04-10,574,b8qh5vii-ni9u-vj2t-v897nnlthl9c
3,59437,7,1,2,2021-01-29,2022-04-10,574,pl4rjqbp-hyh5-8wb4-8s2hsu6gp4q5
4,14404,0,10,5,2021-01-29,2022-04-10,574,1xtme1qr-l07j-oqcl-8jvscpwuqxxc
5,33804,1,11,9,2021-01-29,2022-04-10,574,z4znwbrp-9n8x-4yrk-193zjdhd8esi
6,55346,1,11,11,2021-01-29,2022-04-10,574,pd8vmd6n-vlum-svbr-ndh7rcc4k3gn
7,42820,7,5,9,2021-01-29,2022-04-10,574,um4naita-t2of-erck-b93x65jfi4ku
8,11067,4,11,10,2021-01-29,2022-04-10,574,fdfj9w31-vdi2-agau-k36gq45l6oyv
9,98507,3,10,4,2021-01-29,2022-04-10,574,bf7m3f49-vj7z-z16h-u5cpcely2pt3


In [ ]:
#Table 6

In [17]:
agent_name_id = input("Enter agent_name_id: ")

Enter agent_name_id: 574


In [19]:

cdf = Dataset.withColumn('id',round(rand()*100300,1000).cast("int"))\
    .withColumn("fixed_credit_limit",round(rand()*90,100).cast("int"))\
    .withColumn("credit_limit",round(rand()*12,5).cast("int"))\
    .withColumn("due_amount",round(rand()*120,500).cast("int"))\
    .withColumn('created_date',lit('2020-01-14'))\
    .withColumn('updated_date',current_date())\
    .withColumn('agent_name_id',lit(agent_name_id))\
    .withColumn('rfid_id',randIdGenerator()).drop('Employee Name','Contact','Emp ID')

users_postpaidrfid = cdf.select("id","fixed_credit_limit","credit_limit","due_amount","created_date","updated_date","agent_name_id","rfid_id")
  

In [20]:
users_postpaidrfid.toPandas().head(10)

,id,fixed_credit_limit,credit_limit,due_amount,created_date,updated_date,agent_name_id,rfid_id
0,88470,87,0,72,2020-01-14,2022-04-12,574,8dkuqzik-dttz-tve2-vrjn7jhpyz3z
1,8162,32,6,113,2020-01-14,2022-04-12,574,c28kefbp-dnkh-i6zw-0ypou1skquly
2,19992,5,5,57,2020-01-14,2022-04-12,574,zys5w4ds-fidv-zy4h-a0er7hlg4at9
3,66694,36,10,47,2020-01-14,2022-04-12,574,ddjd0fa9-7fg5-p27j-3nsdcjk20a6z
4,89844,26,7,38,2020-01-14,2022-04-12,574,w4c1ykac-nk3r-c9g8-t683lhgk3v84
5,87676,65,7,69,2020-01-14,2022-04-12,574,x99i505t-lxw8-kojw-cogxsfbkgq50
6,43573,48,7,69,2020-01-14,2022-04-12,574,mjh8yfvi-i397-r8jd-2ddd158l881g
7,28486,23,0,108,2020-01-14,2022-04-12,574,cw5vikp0-3jxh-4f6g-wba9rapse3o8
8,76561,50,3,93,2020-01-14,2022-04-12,574,kmmz1rc9-6e12-n6w7-b18qjaxjr9es
9,14935,56,6,68,2020-01-14,2022-04-12,574,oebqg3xb-se93-k5v3-jg72dyektpjv


In [21]:

#Dumping The created dataset as a csv file.

users_postpaidrfid.write.option("header",True).csv("/home/eternal/bigData/finalProject/DataSet/users_postpaidrfid")

In [22]:

#Opening the created file from folder

datasetTable6 = spark.read.load("/home/eternal/bigData/finalProject/DataSet/users_postpaidrfid/part-00000-c5eb94cd-2410-4296-bc0b-cb9281110316-c000.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )
datasetTable6.toPandas().head(10)

,id,fixed_credit_limit,credit_limit,due_amount,created_date,updated_date,agent_name_id,rfid_id
0,88470,87,0,72,2020-01-14,2022-04-12,574,0s95m8dp-wdj5-smxu-fn7mxmqr6fbm
1,8162,32,6,113,2020-01-14,2022-04-12,574,42wu3gqo-7feo-4k6a-0f13zxohhejt
2,19992,5,5,57,2020-01-14,2022-04-12,574,l4nnrt47-k2uw-dp6p-6pmveusb4pe6
3,66694,36,10,47,2020-01-14,2022-04-12,574,ck44jvr2-zcon-0rqt-api3z2uepqwm
4,89844,26,7,38,2020-01-14,2022-04-12,574,7khqjcut-1yl5-ixes-89ujhsydusc9
5,87676,65,7,69,2020-01-14,2022-04-12,574,6ohby3ni-s7yu-59i2-ktnjsbpis56o
6,43573,48,7,69,2020-01-14,2022-04-12,574,r3u2yu8a-yco0-q6om-411g5zc9uzxy
7,28486,23,0,108,2020-01-14,2022-04-12,574,9ti3h764-oqym-ufse-9bv3mdt3pizf
8,76561,50,3,93,2020-01-14,2022-04-12,574,s6j7ds7f-atvl-qs62-1wybv5ju32mg
9,14935,56,6,68,2020-01-14,2022-04-12,574,3crtm8nm-e17m-dviq-tebsaj6wa01f


In [ ]:
#Table 7

In [91]:

cdf = Dataset.withColumn('user_id',randIdGenerator())\
    .withColumn("per_day",round(rand()*9,5).cast("int"))\
    .withColumn("per_week",round(rand()*12,7).cast("int"))\
    .withColumn("per_month",round(rand()*15,28).cast("int"))\
    .withColumn('created_date',lit('2021-08-17'))\
    .withColumn('updated_date',current_date()).drop('Employee Name',
                                                    'Contact','Emp ID')

users_checkout_limit = cdf.select("user_id","per_day","per_week",
                                  "per_month","created_date","updated_date")
  

In [92]:
users_checkout_limit.toPandas().head(10)

,user_id,per_day,per_week,per_month,created_date,updated_date
0,riyzah7o-8413-xkoe-v5esmotmh0th,2,6,1,2021-08-17,2022-04-06
1,ga7wjx1u-hrca-au1c-5bp93e0v7par,7,2,7,2021-08-17,2022-04-06
2,lnru6cer-copc-ka1b-409w6hgnsx8z,2,7,1,2021-08-17,2022-04-06
3,aahng84s-yxwd-dotq-7pbr7iek046f,5,10,8,2021-08-17,2022-04-06
4,869bm00x-yhr9-0d5n-2sxyam7ci13u,1,2,10,2021-08-17,2022-04-06
5,zitesca9-3pmq-l9id-zg4638utdq1l,2,6,3,2021-08-17,2022-04-06
6,so0ii4vn-pu6y-seae-f0mejn9dx62h,4,6,0,2021-08-17,2022-04-06
7,gnbb1xtn-gigp-rtam-qrvndx477qli,0,9,1,2021-08-17,2022-04-06
8,kuug9cx6-byw8-qs9i-l6pgrs3ngogc,2,1,14,2021-08-17,2022-04-06
9,v24dlg92-9p6u-r6q0-wbif5fx12x60,0,1,3,2021-08-17,2022-04-06


In [93]:
#Dumping The created dataset as a csv file.

users_checkout_limit.write.option("header",True).csv("/home/eternal/bigData/finalProject/DataSet/users_checkout_limit")

In [94]:
#Opening the created file from folder

datasetTable7 = spark.read.load("/home/eternal/bigData/finalProject/DataSet/users_checkout_limit/part-00000-03e0c516-3c72-48d5-9955-cd9b3ffc65fa-c000.csv", format = "csv",sep = ",", inferScheme = True, header = True,minPartitions = 2 )
datasetTable7.toPandas().head(10)

,user_id,per_day,per_week,per_month,created_date,updated_date
0,glsqwzgy-tudc-2r60-ap42fnvgfgjp,2,6,1,2021-08-17,2022-04-06
1,cyh35ham-7q3q-fief-m57ikg069swp,7,2,7,2021-08-17,2022-04-06
2,0q0crb55-jpuh-hd4b-tjajk487dbq7,2,7,1,2021-08-17,2022-04-06
3,6f0pajel-vg8o-m4y7-dc678y6pg4ep,5,10,8,2021-08-17,2022-04-06
4,5kmapnf4-h6rf-lfp5-i565a1nqj3yi,1,2,10,2021-08-17,2022-04-06
5,uvuqzm2u-c48v-om30-vk46zahkqgb4,2,6,3,2021-08-17,2022-04-06
6,yqb344a8-ffuo-3jno-21bqhj11b7h0,4,6,0,2021-08-17,2022-04-06
7,xtuabkxq-3lqx-nair-ewsg8qv12902,0,9,1,2021-08-17,2022-04-06
8,t5628nmg-f4sy-b312-85nesbr6ssrj,2,1,14,2021-08-17,2022-04-06
9,2yyked5b-4xvt-28u1-h895bxngrghs,0,1,3,2021-08-17,2022-04-06


In [ ]:
# yay  Project Complete... :-)

In [2]:
#Change font size

%%html
<style type='text/css'>
.CodeMirror{
    font-size: 21px;
}

div.output_area pre {
    font-size: 21px;
}
</style>